# Worksheet 07

Name:  Tian Tan
UID: U68023245

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

In [24]:

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.00454091033587, 5.127170528762501, 6.083750614945755, 2.6909115900721305, 4.9597176401197975, 4.027602374601293, 3.6781615047787914, 5.125851153489565, 4.973032005880094, 4.182078692808478]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [25]:
c2 = np.random.normal(8, 1, 10).tolist()
print(c2)

[6.168324468415767, 9.531698441762249, 9.258256418787441, 5.699593008130197, 8.552171606471628, 8.298650346970508, 8.179978891949647, 8.985073398638365, 8.491047233537694, 7.823228229914113]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [26]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[7.823228229914113, 8.491047233537694, 4.182078692808478, 4.973032005880094, 5.125851153489565, 8.985073398638365, 8.179978891949647, 3.6781615047787914, 8.298650346970508, 4.027602374601293]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.


P(c1) = 1/2 = P(c2)

mu_1 = 5, mu_2 = 8 

var_1 = var_2 = 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [27]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2)/ (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2)]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.182078692808478, 4.973032005880094, 5.125851153489565, 3.6781615047787914, 4.027602374601293]
[7.823228229914113, 8.491047233537694, 8.985073398638365, 8.179978891949647, 8.298650346970508]
P(C_1) = 0.5,  P(C_2) = 0.5
mean_1 = 4.397345146311644,  mean_2 = 8.355595620202063
var_1 = 0.3124821672516878,  var_2 = 0.14641769014556585


c:\Users\tt2ma\.conda\envs\CS506HMK1\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


P(C_1) and P(C_2) are both 0.5. This makes sense. They are close to the true values.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [28]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    #print("point = ", p)
    pdf_i = [] # two values (two cluasters)

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        # print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j] 

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0]) # data
    print("Probability of coming from C_1 = " + str(p[1])) # prob_c0_x
    print("Probability of coming from C_2 = " + str(p[2])) # prob_c1_x
    print()


7.823228229914113
Probability of coming from C_1 = 2.7604393948224823e-24
Probability of coming from C_2 = 1.0

8.491047233537694
Probability of coming from C_1 = 3.87782100985865e-38
Probability of coming from C_2 = 1.0

4.182078692808478
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 1.0060835409802024e-176

4.973032005880094
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 1.4884201217847957e-115

5.125851153489565
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 7.098891489074061e-105

8.985073398638365
Probability of coming from C_1 = 7.560701658553004e-44
Probability of coming from C_2 = 1.0

8.179978891949647
Probability of coming from C_1 = 1.45789921625505e-32
Probability of coming from C_2 = 1.0

3.6781615047787914
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 7.463102852656092e-221

8.298650346970508
Probability of coming from C_1 = 7.183575316917331e-35
Probability of coming from C_2 = 

Based on estimated probabilities, 

C1: [4.182078692808478, 4.973032005880094, 5.125851153489565, 3.6781615047787914, 4.027602374601293]

C2: [7.823228229914113, 8.491047233537694, 8.985073398638365, 8.179978891949647, 8.298650346970508]


Compared with the true value, the estimated clusters are very close to the true clusters and they are actually the same.


f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [33]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum(map(lambda x : x[0]* (x[1] - mean[0])**2, zip(prob_c0_x, data))) / sum(prob_c0_x), sum(map(lambda x : x[0]*(x[1] - mean[1])**2, zip(prob_c1_x, data))) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.5,  P(C_2) = 0.5
mean_1 = 4.397345146311644,  mean_2 = 8.355595620202063
var_1 = 0.3124821672516878,  var_2 = 0.14641769014556585


Following are the true values:

P(C_1) = 0.5,  P(C_2) = 0.5

mean_1 = 4.397345146311644,  mean_2 = 8.355595620202063

var_1 = 0.3124821672516878,  var_2 = 0.14641769014556585

In my case, these estimated values are same with the original ones got from K means. Generally, I think they should be quite similar.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [37]:
# Using above estimated values (mean...) to update P(C_j, X_i)

prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum(map(lambda x : x[0]* (x[1] - mean[0])**2, zip(prob_c0_x, data))) / sum(prob_c0_x), sum(map(lambda x : x[0]*(x[1] - mean[1])**2, zip(prob_c1_x, data))) / sum(prob_c1_x)]
# print(mean, var, prob_c)

# refreash parameters
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    pdf_i = [] # two values (two cluasters)

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        prob_c[j] 

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
# for p in probs:
#     print(p[0]) # data
#     print("Probability of coming from C_1 = " + str(p[1])) # prob_c0_x
#     print("Probability of coming from C_2 = " + str(p[2])) # prob_c1_x
#     print()

# display the new parameters after update P(C_j, X_i)
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum(map(lambda x : x[0]* (x[1] - mean[0])**2, zip(prob_c0_x, data))) / sum(prob_c0_x), sum(map(lambda x : x[0]*(x[1] - mean[1])**2, zip(prob_c1_x, data))) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.5,  P(C_2) = 0.5
mean_1 = 4.397345146311644,  mean_2 = 8.355595620202063
var_1 = 0.3124821672516878,  var_2 = 0.14641769014556585


In my case, there is no difference between these two iterations. The reason is that at the first iteration, it has converged and the best result is obtained. However, in my other test cases, by updating P(C_j, X_i), the estimated values can be brought closer to the true values. And it will eventually converge.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [49]:

# hard assignment either 0 or 1

# print(data)
label = {}

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    # p[0]: data; p[1]: prob_c0_x; p[2]: prob_c1_x
    # if prob_c0_x > prob_c1_x -> cluster 0
    if p[1] > p[2]:
        label[p[0]] = 0
    else:
        label[p[0]] = 1
    print("point: " + str(p[0]) + " belongs to cluster: " + str(label[p[0]]))
# print(label)



point: 7.823228229914113 belongs to cluster: 1
point: 8.491047233537694 belongs to cluster: 1
point: 4.182078692808478 belongs to cluster: 0
point: 4.973032005880094 belongs to cluster: 0
point: 5.125851153489565 belongs to cluster: 0
point: 8.985073398638365 belongs to cluster: 1
point: 8.179978891949647 belongs to cluster: 1
point: 3.6781615047787914 belongs to cluster: 0
point: 8.298650346970508 belongs to cluster: 1
point: 4.027602374601293 belongs to cluster: 0


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      disagree       |
| A  C |      agree       |
| A  D |      agree       |
| A  E |      agree       |
| B  C |      disagree       |
| B  D |      agree       |
| B  E |      agree       |
| C  D |      agree       |
| C  E |      agree       |
| D  E |      disagree       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

It is C(N,2) = N! / (2!*(N-2)!). It means N choose 2, that is, N points can have this number of unique pairs.

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

Since there are 9 points. C(9, 2) = 36. Thus the maximum number of disagreements is 36.

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

cluster 0: 3 agreements.

cluster 1：1 agreements.

cluster 2: 6 agreements.

6+1+3=10， there are 10 agreements in total.

36-10 = 26. Thus, there are 26 disagreements between C and P.

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

Given a N points dataset, as we known, the maximum number of disagreements is C(N, 2).

Since filering the dataset by cluster number is a easy operation, then we can easily split our data into several clusters. Then we can compute the number of agreements of each cluster, it should be easy.

Finally, we can just use the the maximum number of disagreements minus the sum of the number of agreements of each cluster to get the disagreements distance on this datasets.

That is,

disagreements distance = C(N,2) - sum(ni)

ni represents the agreements number of each cluster.